In [1]:
%load_ext nb_black
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#%matplotlib inline;
import ast
import json
from pandas.io.json import json_normalize

import datetime
import time
import math

from scipy import stats
from scipy import signal as spsg

METADATA = "../../metadata/"
SENSORDATA = "../../sensordata/"

warnings.simplefilter(action="ignore", category=FutureWarning)

def get_derivative(df, timedelta, d=1):

    """sensordata로 구성된 pandas series or dataframe과 timedelta(측정 시간 간격)을 받아 미분 값을 반환한다. 

        Args:
            df: sensordata (timeseries) df or series
            timedelta: dt
            d: df dimension
            
        Returns:
            dataframe or series of derivative of df
        """

    if d == 2:
            return pd.DataFrame(
                np.gradient(df, timedelta, axis=0),
                columns=["Palm", "Index", "Middle", "Ring", "Pinky"],
            )
        else:
            return pd.Series(np.gradient(df, timedelta, axis=0).flatten())

def butterworth_filter(order, cutoff, timedelta):

    """Butterworth lowpass filter for high frequency noise & outliers (mostly sensor error)
    
        Args:
            order: order of filter
            cutoff: threshold frequency
            timedelta: dt
            
        Returns:
            dataframe or series of derivative of df
    """

    sampling_rate = 1 / timedelta
    wn = cutoff / (sampling_rate * 0.5)
    # Nyquist frequency = 0.5 * sampling rate
    b, a = spsg.butter(order, wn, btype="lowpass")
    print("Butterworth Filter: Order n = ", order, "Cutoff ", cutoff, "Hz")

    return b, a

def reject_outliers(data, types):

    """시스템 상에서 센서데이터 처리 방식 오류로 인한 에러 처리
    속도 기준 720 deg/s, 가속도 기준 21600 deg/s^2 넘을 시 interpolation 값으로 대체
    
        Args:
            data: dataframe OR series of speed OR acc
            types: speed OR acceleration
            
        Returns:
            dataframe or series with interpolated values for outliers
    """

    if types == "speed":
        threshold = 720
    elif types == "acceleration":
        threshold = 21600
    data = np.asarray(data)
    data = np.where(np.abs(data) < threshold, data, np.nan)
    data[np.isnan(data)] = np.interp(
        np.isnan(data).nonzero()[0],
        np.isfinite(data).nonzero()[0],
        data[np.isfinite(data)],
    )

    return data

def zero_crossing(my_array, threshold):

    """Count zero crossings of value with threshold. 
    Value should not only change its sign but its change should exceed threshold
    
        Args:
            my_array: dataframe OR series OR numpy array of speed OR acc
            threshold: threshold value
            
        Returns:
            Count of zerocrossings over threshold 
    """

    my_array = np.asarray(my_array)
    return (
        ((my_array[:-1] * my_array[1:]) < 0)
        & (abs(my_array[:-1] - my_array[1:]) > threshold)
    ).sum()

def check_treasure_range(window, ranges):

    """주어진 Pandas Series 또는 DataFrame type의 데이터를 미분(정확히는 gradient)한다. 
    
    Args:
        df: 미분하고자 하는 pandas series OR dataframe
        timedelta: 미분하고자 하는 데이터셋의 시간간격(단위: 초)
        d: 미분해야하는 column이 여러 개일 경우 True (e.g. Finger F/E 컨텐츠는 5개 column이 함께 계산되어야 함)
        
    Returns:
        Input에 따라 미분된 Pandas Series 또는 DataFrame 
    """

    
    window_ls = [min(window), max(window)]

    range_ls = []
    for tp in ranges:
        range_ls.append(tp[0])
        range_ls.append(tp[1])
    range_ls

    check = np.digitize(window_ls, bins=range_ls)

    if check[0] == check[1] and check[0] % 2 == 1:
        return True
    else:
        return False

class EMR:

    """훈련결과 인스턴스.
    
    유저가 수행완료한 훈련컨텐츠에 대한 결과 정보를 담고 있다.
    센서데이터 단위에서 도출가능한 정보 및 각종 바이오마커가 본 인스턴스를 통해 계산되고 시각화된다.
    
    Attributes:
        mid: integer. mission id. in Live or Alpha DB.
        uid2: string. unique string with length 2 in case of multiple EMR for the same mission id.
        pid: integer. patient id.
        cid: string. content name. (e.g. 'SMART_GLOVE_FISHING')
        tt: string. training type. (e.g. 'SMART_GLOVE_FOREARM_SUP_PRONATION')
        range: list. ROM used for the gameplay, calculated by {calibration range} * {arom ratio}
        effective_range: list. ROM that are counted as a motion success in the gameplay. depends on contents.
        treatment: True/False. True for actual user (patients), False for test user.
        patient_info: list. patient's info in order of [{nickname}, {birthday}, {gender}] from Live DB.
        time: string. timestamp when emr generated
        timedelta: float. average dt of sensordata (seconds)
        playtime: int. total played time
        num_lackpeak: int. number of peaks user have made but failed to maintain 5 seconds.
    """

    def __init__(self, mission_id, treatment = True):

        """EMR 생성자.

        Args:
            mission_id: integer. mission id. in Live or Alpha DB.
            treatment: True/False. True for actual user (patients), False for test user.

        Raises:
            입력한 mission id에 해당하는 훈련결과가 여러 개일 경우, 
            각 훈련결과 정보(훈련일자 및 시간, 훈련소요시간)를 출력한 후 uid2를 입력받는다.
        """

        content = pd.read_csv(METADATA + "content_training_type.csv")
        # Class variable: content와 training type matching을 갖고 있는 테이블

        self.mid = mission_id
        self.treatment = treatment

        # if patient,
        if treatment == 1:

            patient = pd.read_csv(METADATA + "p1_patient_list.csv")

            match = pd.read_csv(METADATA + "p1_mission_list.csv")
            match.dropna(subset=["uid2"], inplace=True)
            match["calibration_range"] = match["calibration_range"].apply(
                ast.literal_eval
            )
            match.uid2.apply(str)

            # mission id가 같은 exercise_mission_result가 여러 개일 경우 별도 unique id를 입력
            if len(match.loc[match["mission_id"] == mission_id]) > 1:
                print(
                    "There are multiple exercise mission results:",
                    match.loc[match["mission_id"] == mission_id][
                        [
                            "exercise_mission_result_generated_time",
                            "played_time",
                            "uid2",
                        ]
                    ],
                )
                self.uid2 = str(input("Pick one of those and input uid2"))
                idx = match.loc[
                    (match["mission_id"] == mission_id) & (match["uid2"] == self.uid2)
                ].index.item()
            else:
                idx = match.loc[match["mission_id"] == mission_id].index.item()
                self.uid2 = match.loc[idx, "uid2"]

            self.pid = match.loc[idx, "patient_id"]
            self.patient_info = patient.loc[
                patient["patient_id"] == self.pid,
                ["patient_nickname", "patient_birthday", "patient_gender"],
            ].values

        # if general,
        elif treatment == 0:

            self.patient_info = "Control group"

            match = pd.read_csv(METADATA + "g1_mission_list.csv")
            match["calibration_range"] = match["calibration_range"].apply(
                ast.literal_eval
            )

            if len(match.loc[match["mission_id"] == mission_id]) > 1:
                idx = match.loc[
                    match["mission_id"] == mission_id, "played_time"
                ].idxmax()
            else:
                idx = match.loc[match["mission_id"] == mission_id].index.item()

            self.pid = 0
            self.uid2 = "00"

        self.time = match.loc[idx, "exercise_mission_result_generated_time"]
        self.cid = match.loc[idx, "content_id"]
        self.tt = content.loc[
            content["content_id"] == match.loc[idx, "content_id"], "training_type_name"
        ].item()
        self.playtime = match.loc[idx, "played_time"]
        self.arom_ratio = match.loc[idx, "arom_ratio"]

        # Get calibration range (게임컨텐츠 내에서 성공 여부를 판단하는 ROM 계산)
        ls = list(match.loc[idx, "calibration_range"].values())
        if len(ls) == 1:
            calrange = abs(ls[0][1] - ls[0][0])
            rangemax = max(ls[0][1], ls[0][0]) - (
                (calrange * (1 - self.arom_ratio)) / 2
            )
            rangemin = min(ls[0][1], ls[0][0]) + (
                (calrange * (1 - self.arom_ratio)) / 2
            )
            self.range = [rangemin, rangemax]
        else:
            self.range = ls

    def effective_range(self):

        """실제 훈련에서 유효한 ROM 범위를 계산한다.
        
        훈련종류에 따라 동작성공에 해당하는 ROM 범위가 다르기 때문에 
        calibration을 통해 수집된 ROM을 훈련종류에 맞는 수식을 통해 유효 ROM 범위를 계산한다.
        """

        
        game_rom_minmax = ['SMART_GLOVE_POUR_WINE', 'SMART_GLOVE_FISHING']
        game_coordination_div8 = ['SMART_GLOVE_FIND_TREASURE']
        game_rom_finger = ['SMART_GLOVE_MAKE_JUICE']
        #add game lists and games by appropriate categorization
        
        if self.cid in game_rom_minmax:
            return self.range
        elif self.cid in game_coordination_div8:
            ls = [(-40) + 80 * i / 7 for i in range(8)]
            ls_final = [
                (
                    (
                        (self.range[1] - self.range[0])
                        * (j - 360 * np.arcsin(50 / 1053) / np.pi)
                        / 80
                    )
                    + (self.range[1] + self.range[0]) / 2,
                    (
                        (self.range[1] - self.range[0])
                        * (j + 360 * np.arcsin(50 / 1053) / np.pi)
                        / 80
                    )
                    + (self.range[1] + self.range[0]) / 2,
                )
                for j in ls
            ]
            return ls_final
        elif self.cid in game_rom_finger:
            return self.range
        else:
            return self.range

    def show_info(self):

        """해당 exercise mission result에 대한 기본 정보를 출력한다.
        
        """

        print("Info of Mission ID #", self.mid)
        print("Exercise Mission Result Unique id:", self.uid2)
        print("Patient id:", self.pid)
        print("Patient info:", self.patient_info)
        print("Content name:", self.cid)
        print("Exercise time:", self.time)
        print("Training type:", self.tt)
        print("Calibration range:", self.range)

    def get_df(self, sub = False):

        """주어진 Pandas Series 또는 DataFrame type의 데이터를 미분(정확히는 gradient)한다. 

        Args:
            sub: T/F. 훈련에 쓰이는 센서데이터 외 센서데이터 사용 여부 (e.g. POUR_WINE 에서 trans Forearm S/P 값 외의 센서값 사용 시 True 입력)
            
        Returns:
            Pandas Series/DataFrame. 해당 EMR에 맞는 sensordataset (if sub == False).
            (if sub == True, 입력값에 따른 sensordataset 반환)
            
        Raises:
            if sub == True, 해당 EMR에 맞는 sensordata 외로 어떤 sensordata를 받을 것인지 입력한다. 
        """

        if self.treatment == 1:

            df = pd.read_csv(
                SENSORDATA + "p1/p1_" + str(self.mid) + "_" + self.uid2 + ".csv",
                thousands=",",
            )
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            df.sort_values(by=["timestamp"], inplace=True)
            df.reset_index(drop=True, inplace=True)

        elif self.treatment == 0:

            df = pd.read_csv(
                SENSORDATA + "g1/g1_" + str(self.mid) + ".csv", thousands=","
            )
            df.rename(columns=lambda x: x.replace("actionData.", ""), inplace=True)
            df["timestamp"] = df["@timestamp"].apply(
                lambda x: datetime.datetime.strptime(x, "%b %d, %Y @ %H:%M:%S.%f")
            )
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            df.sort_values(by=["timestamp"], inplace=True)
            df.reset_index(drop=True, inplace=True)

        self.timedelta = 0.001 * np.mean(
            np.gradient(df["timestamp"]).astype("timedelta64[ms]").astype("float")
        )

        if any(
            np.gradient(df["timestamp"]).astype("timedelta64[ms]").astype("float")
            > 3000
        ) or ((self.playtime * 30) > len(df)):
            raise ValueError(
                "Possibility of missing data in EMR of ",
                self.mid,
                " by \n",
                "1) gap of ",
                0.001
                * max(
                    np.gradient(df["timestamp"])
                    .astype("timedelta64[ms]")
                    .astype("float")
                ),
                " seconds, \n",
                "2) total about ",
                ((self.playtime * 30) - len(df)) / 33,
                " seconds",
            )

        if sub == 0:
            cal_avg = 0
            if self.tt == "SMART_GLOVE_FOREARM_SUP_PRONATION":
                i = 0
                self.trans = 1
            elif self.tt == "SMART_GLOVE_WRIST_FLEX_EXT_AG":
                i = 1
                self.trans = 1
            elif self.tt == "SMART_GLOVE_WRIST_DEVIATION_EG":
                i = 2
                self.trans = 1
            elif self.tt == "SMART_GLOVE_FINGER_FLEX_EXT":
                i = 3
                self.trans = 0
            else:
                "Contents that has not yet developed for biomarker search!"

            if self.trans == 1:
                ls = [
                    "wristPStrans",
                    "wristFEtrans",
                    "wristUDRDtrans",
                    [
                        "fingerPalmtrans",
                        "fingerIndextrans",
                        "fingerMiddletrans",
                        "fingerRingtrans",
                        "fingerPinkytrans",
                    ],
                ]
            else:
                ls = [
                    "wristPS",
                    "wristFE",
                    "wristUDRD",
                    [
                        "fingerPalm",
                        "fingerIndex",
                        "fingerMiddle",
                        "fingerRing",
                        "fingerPinky",
                    ],
                ]

            if i == 3:
                ls = ls[i]
                ls.insert(0, "timestamp")
            else:
                ls = ["timestamp", ls[i]]

            df = df[ls]

        else:
            cal_avg = 0
            dic = {
                1: "wristPS",
                2: "wristFE",
                3: "wristUDRD",
                4: [
                    "fingerPalm",
                    "fingerIndex",
                    "fingerMiddle",
                    "fingerRing",
                    "fingerPinky",
                ],
                5: "wristPStrans",
                6: "wristFEtrans",
                7: "wristUDRDtrans",
                8: [
                    "fingerPalmtrans",
                    "fingerIndextrans",
                    "fingerMiddletrans",
                    "fingerRingtrans",
                    "fingerPinkytrans",
                ],
            }
            print(pd.DataFrame.from_dict(dic, orient="index"))
            query = input(
                "Select columns to export by index (press 0 to select all):"
            ).split()

            if query == ["0"]:
                df = df.iloc[:, np.r_[2, 5:21]]
                ls = list(df.columns()) + ["fingerAvg"]
                cal_avg += 1
            else:
                ls = ["timestamp"]
                for i in query:
                    if int(i) == 4:
                        ls = ls + dic[int(i)]
                    elif int(i) == 8:
                        ls = ls + dic[int(i)]
                        cal_avg += 1
                    else:
                        ls.append(dic[int(i)])
                df = df[ls]

        if cal_avg == 1:
            df["fingerAvg"] = df[dic[8]].apply(np.mean, axis=1)
            ls.append("fingerAvg")
        else:
            pass

        df = df.dropna(axis=0).reset_index(drop = True)

        return df, ls

    def show_graph(self, sub = False, peak=0, save = False):

        """Visualize sensordata as a plot (seaborn). 

        Args:
            sub: T/F. 훈련에 쓰이는 센서데이터 외 센서데이터 사용 여부 (e.g. POUR_WINE 에서 trans Forearm S/P 값 외의 센서값 사용 시 True 입력)
            peak: int. peak detection algorithm 선택.
            save: T/F. 저장 여부

        Returns:
            seaborn plot. 
            (if save == True, html file named '{patient id}_{mission_id}.png' is saved under folder named 'images'
        """

        df, ls = self.get_df(sub=sub)
        df_plot = pd.melt(df, id_vars=["timestamp"], value_vars=ls[1:])

        # plot
        fig = plt.figure(figsize=(40, 20))
        sns.set(style="ticks", palette="muted", font_scale=1.8)
        if self.treatment == 1:
            plt.suptitle(
                "Patient#: "
                + str(self.pid)
                + "\n Content: "
                + str(self.cid)[12:]
                + " ("
                + str(self.tt)[12:]
                + ")"
                + "\n Mission#: "
                + str(self.mid)
            )
        elif self.treatment == 0:
            plt.suptitle(
                "Control Group with Content: "
                + str(self.cid)[12:]
                + " ("
                + str(self.tt)[12:]
                + ")"
                + "\n Mission#: "
                + str(self.mid)
            )
        if "fingerAvg" in ls:
            fig = plt.figure(figsize=(40, 40))
            ax1 = fig.add_subplot(211)
            ax2 = fig.add_subplot(212)

            sns.lineplot(
                data=df_plot,
                x="timestamp",
                y="value",
                hue="variable",
                dashes=False,
                ax=ax1,
            )
            sns.lineplot(data=df, x="timestamp", y="fingerAvg", ax=ax2)

            ax1.set(ylabel="Value", title="For each finger")
            ax2.set(xlabel="Value", title="Average of fingers")
        else:
            sns.lineplot(
                x="timestamp", y="value", data=df_plot, hue="variable", dashes=False
            )

        if sub == 0:
            if self.trans == 1:
                plt.axhspan(self.range[0], self.range[1], facecolor="k", alpha=0.05)

        if peak == 1:
            df_peak = self.detect_peak()
        elif peak == 2:
            df_peak = self.detect_final_peak()
        elif peak == 3:
            df_peak = self.detect_modified_peak()
        elif peak == 4:
            df_peak = self.detect_modified_s_peak()
        else:
            df_peak = []

        if len(df_peak) < 1:
            print("No detected peak")
        else:
            for index, value in df_peak.iteritems():
                plt.axvspan(
                    df.loc[value[0], "timestamp"],
                    df.loc[value[-1], "timestamp"],
                    facecolor="skyblue",
                    alpha=0.2,
                )

        if save == 1:
            plt.savefig(
                "../../images/" + str(self.pid) + "_" + str(self.mid) + ".png",
                quality=50,
            )
        else:
            pass

    def show_plotly(self, sub = False, peak = 0, save = False):

        """Visualize sensordata as an interactive plot (plotly). 

        Args:
            sub: T/F. 훈련에 쓰이는 센서데이터 외 센서데이터 사용 여부 (e.g. POUR_WINE 에서 trans Forearm S/P 값 외의 센서값 사용 시 True 입력)
            peak: int. peak detection algorithm 선택.
            save: T/F. 저장 여부

        Returns:
            Plotly plot. 
            (if save == True, html file named '{patient id}_{mission_id}.html' is saved under folder named 'images'
        """

        df, ls = self.get_df(sub=sub)
        print(ls)

        template = "plotly_white"
        if self.tt == "SMART_GLOVE_FINGER_FLEX_EXT":

            fig = go.Figure(layout=go.Layout(hovermode="x"))
            fig.add_trace(
                go.Scatter(x=df.timestamp, y=df[ls[1]], mode="lines", name="Palm")
            )
            fig.add_trace(
                go.Scatter(x=df.timestamp, y=df[ls[2]], mode="lines", name="Index")
            )
            fig.add_trace(
                go.Scatter(x=df.timestamp, y=df[ls[3]], mode="lines", name="Middle")
            )
            fig.add_trace(
                go.Scatter(x=df.timestamp, y=df[ls[4]], mode="lines", name="Ring")
            )
            fig.add_trace(
                go.Scatter(x=df.timestamp, y=df[ls[5]], mode="lines", name="Pinky")
            )
            fig.update_layout(
                shapes=[
                    go.layout.Shape(
                        type="rect",
                        x0=df["timestamp"].min(),
                        y0=self.range[0],
                        x1=df["timestamp"].max(),
                        y1=self.range[1],
                        fillcolor="whitesmoke",
                        opacity=0.25,
                        layer="below",
                        line_width=0,
                    )
                ]
            )

        else:
            fig = px.line(df, x="timestamp", y=ls[1], template=template)
            fig.update_layout(
                shapes=[
                    go.layout.Shape(
                        type="rect",
                        x0=df["timestamp"].min(),
                        y0=self.range[0],
                        x1=df["timestamp"].max(),
                        y1=self.range[1],
                        fillcolor="whitesmoke",
                        opacity=0.25,
                        layer="below",
                        line_width=0,
                    )
                ]
            )

        figname = "../images/" + str(self.pid) + "_" + str(self.mid) + ".html"

        if self.treatment == 1:
            fig.update_layout(
                title_text="Patient#: "
                + str(self.pid)
                + "\n Content: "
                + str(self.cid)[12:]
                + "\n Mission#: "
                + str(self.mid),
                template=template,
            )
        elif self.treatment == 0:
            fig.update_layout(
                title_text="Control Group with Content: "
                + str(self.cid)[11:]
                + "\n Mission#: "
                + str(self.mid),
                template=template,
            )
        fig.show()

        if save == 1:
            fig.write_html(figname)
        else:
            pass

    def get_speed(self, sub=False, filtering=False):

        """센서데이터를 기반으로 한 속도데이터셋을 생성한다. 

        Args:
            sub: T/F. 훈련에 쓰이는 센서데이터 외 센서데이터 사용 여부 (e.g. POUR_WINE 에서 trans Forearm S/P 값 외의 센서값 사용 시 True 입력)
            filtering: T/F. butterworth low pass filter 적용 여부

        Returns:
            Pandas Series/DataFrame. sensordata를 한번 미분한 속도(IMU: 각속도, Bending sensor: 굽힘 속도) 데이터. 
        
        Raises:
            
        """

        df, ls = self.get_df(sub=sub)

        # Calculate approximate dt for sampling rate (actual value will have its own distribution, but approximate is enough)
        timedelta = 0.001 * np.mean(
            np.gradient(df["timestamp"]).astype("timedelta64[ms]").astype("float")
        )

        if len(ls) == 2:
            d = False
        elif len(ls) > 2:
            d = True
        else:
            raise ValueError(
                "Number of columns in EMR dataframe is ambiguous. Check it"
            )

        if filtering == 0:
            data = get_derivative(df[ls[1:]].dropna(axis=0), timedelta=timedelta, d=d)

        elif filtering == 1:
            b, a = butterworth_filter(5, 5, timedelta)

            if d:
                data = get_derivative(
                    df[ls[1:]].dropna(axis=0), timedelta=timedelta, d=d
                )
                for i in data.columns:
                    data[i] = spsg.filtfilt(b, a, data[i])
            else:
                data = pd.Series(
                    spsg.filtfilt(
                        b,
                        a,
                        get_derivative(
                            df[ls[1:]].dropna(axis=0), timedelta=timedelta, d=d
                        ),
                    )
                )

        return data

    def show_speed(self, sub=False, filtering = False, peak=0):

        """Visualize speed data as a plot (seaborn). 

        Args:
            sub: T/F. 훈련에 쓰이는 센서데이터 외 센서데이터 사용 여부 (e.g. POUR_WINE 에서 trans Forearm S/P 값 외의 센서값 사용 시 True 입력)
            filtering = 
            peak: int. peak detection algorithm 선택.

        Returns:
            seaborn plot. 
        """

        df_speed = self.get_speed(sub=sub, filtering=filtering)

        # Plot
        plt.figure(figsize=(30, 15))
        if self.treatment == 1:
            plt.title(
                "Speed Plot of Patient#: "
                + str(self.pid)
                + "\n Content: "
                + str(self.cid)[12:]
                + "\n Mission#: "
                + str(self.mid)
            )
        elif self.treatment == 0:
            plt.title(
                "Speed Plot of Control Group with Content: "
                + str(self.cid)[11:]
                + "\n Mission#: "
                + str(self.mid)
            )

        if peak == 1:
            df_peak = self.detect_peak()
        elif peak == 2:
            df_peak = self.detect_modified_peak()
        else:
            df_peak = []

        if len(df_peak) < 1:
            print("No detected peak")
        else:
            for index, value in df_peak.iteritems():
                # print(row)
                plt.axvspan(value[0], value[-1], facecolor="c", alpha=0.1)

        sns.lineplot(data=df_speed, dashes=False)

    def get_acceleration(self, sub=0, filtering=0):

        """Visualize speed data as a plot (seaborn). 

        Args:
            sub: T/F. 훈련에 쓰이는 센서데이터 외 센서데이터 사용 여부 (e.g. POUR_WINE 에서 trans Forearm S/P 값 외의 센서값 사용 시 True 입력)
            filtering = 
            peak: int. peak detection algorithm 선택.

        Returns:
            seaborn plot. 
        """
        """
        - 해당 exercise mission result의 sensordata를 미분한 속도 값을 시각화한다. <br> (x축: timestamp, y축: 해당 mission에 알맞은 training type에 해당하는 sensordata의 미분값(속도)) <br> *note: 미분은 np.gradient() 함수 이용, 위 get_derivative() 함수 참고*
        - input: Mission ID, 환자(control, 'p' 혹은 1)/일반인(experiment, 'g' or 0) 여부
        - output: 
        - Plot (seaborn): 
            - (Blue) with no filter
            - (Red) with Butterworth lowpass filter with certain order (now 5) and cutoff freq (now 5) 
        - 해당 mission id 및 환자/일반인에 대한 기본 정보 출력(print): called by get_context()
        """

        df_speed = self.get_speed(sub=sub, filtering=filtering)

        # Calculate approximate dt for sampling rate (actual value will have its own distribution, but approximate is enough)
        timedelta = self.timedelta

        if isinstance(df_speed, pd.Series):
            d = False
        elif len(df_speed.columns()) >= 2:
            d = True
        else:
            raise ValueError(
                "Number of columns in EMR dataframe is ambiguous. Check it"
            )

        if filtering == 0:
            data = get_derivative(df_speed.dropna(axis=0), timedelta=timedelta, d=d)

        elif filtering == 1:

            b, a = butterworth_filter(5, 5, timedelta)

            if d:
                data = get_derivative(df_speed.dropna(axis=0), timedelta=timedelta, d=d)
                for colname in data.columns:
                    data[colname] = spsg.filtfilt(b, a, data[colname])
            else:
                data = pd.Series(
                    spsg.filtfilt(
                        b,
                        a,
                        get_derivative(
                            df_speed.dropna(axis=0), timedelta=timedelta, d=d
                        ),
                    )
                )

        return data

    def show_acceleration(self, sub=False, filtering=False, peak=0):

        """Visualize acceleration data as a plot (seaborn). 

        Args:
            sub: T/F. 훈련에 쓰이는 센서데이터 외 센서데이터 사용 여부 (e.g. POUR_WINE 에서 trans Forearm S/P 값 외의 센서값 사용 시 True 입력)
            filtering = 
            peak: int. peak detection algorithm 선택.

        Returns:
            seaborn plot. 
        """

        acceleration_data = self.get_acceleration(sub=sub, filtering=filtering)

        # Plot
        plt.figure(figsize=(30, 15))
        if self.treatment == 1:
            plt.title(
                "Acceleration Plot of Patient#: "
                + str(self.pid)
                + "\n Content: "
                + str(self.cid)[12:]
                + "\n Mission#: "
                + str(self.mid)
            )
        elif self.treatment == 0:
            plt.title(
                "Acceleration Plot of Control Group with Content: "
                + str(self.cid)[11:]
                + "\n Mission#: "
                + str(self.mid)
            )

        if peak == 1:
            df_peak = self.detect_peak()
        elif peak == 2:
            df_peak = self.detect_modified_peak()
        else:
            df_peak = []

        if len(df_peak) < 1:
            print("No detected peak")
        else:
            for index, value in df_peak.iteritems():
                # print(row)
                plt.axvspan(value[0], value[-1], facecolor="c", alpha=0.1)

        sns.lineplot(data=acceleration_data, dashes=False)

    def detect_peak(self, time=168):

        """Detect peak algorithm. 

        Args:
            time: maintain time to success motion. (5 seconds in general ~ 168 data points by experience)

        Returns:
            seaborn plot. 
        """

        if (
            self.tt == "SMART_GLOVE_FOREARM_SUP_PRONATION"
            or self.tt == "SMART_GLOVE_WRIST_FLEX_EXT_AG"
        ):
            df, ls = self.get_df()

            meta = []
            window = []
            stack_val = []
            stack_idx = []
            num_lackpeak = 0  # calibration range는 넘었으나 5초 유지를 못한 경우 count

            for row in df.itertuples():
                if len(window) < 2:
                    # window에 element가 2개가 채워질 때까지는 그냥 채움
                    window.append(row[2])
                    if len(window) == 2:
                        # window에 element가 2개가 되고 마지막 element가 ROM calrange 바깥에 있으면 stack하기 시작
                        if window[1] >= self.range[1] or window[1] <= self.range[0]:
                            stack_val.append(row[2])
                            stack_idx.append(row.Index)
                else:
                    del window[0]
                    window.append(row[2])
                    if (window[0] >= self.range[0]) and (self.range[0] >= window[1]):
                        # Start Stacking (Case A)
                        if len(stack_val) == len(stack_idx) == 0:
                            stack_val.append(row[2])
                            stack_idx.append(row.Index)

                        else:
                            print(
                                "Something is wrong (start 1)",
                                self.mid,
                                window,
                                row.Index,
                                df.loc[row.Index, "timestamp"],
                            )

                            if abs(window[1] - window[0]) > (
                                self.range[1] - self.range[0]
                            ):
                                print("Sensor Error")
                                stack_val.append(row[2])
                                stack_idx.append(row.Index)
                                if len(stack_idx) >= time:
                                    meta.append([stack_idx[0], stack_idx[-1]])
                                else:
                                    pass
                                stack_val = [row[2]]
                                stack_idx = [row.Index]
                            else:
                                print("Fuck, what is this error?")

                    elif (window[0] <= self.range[0]) and (self.range[0] <= window[1]):
                        # Stop Stacking (Case B)
                        if (len(stack_val) != 0) or (len(stack_idx) != 0):
                            stack_val.append(row[2])
                            stack_idx.append(row.Index)
                            if len(stack_idx) >= time:
                                meta.append([stack_idx[0], stack_idx[-1]])
                            else:
                                num_lackpeak += 1
                            stack_val = []
                            stack_idx = []
                        else:
                            print(
                                "Something is wrong (stop 1)",
                                self.mid,
                                window,
                                row.Index,
                                df.loc[row.Index, "timestamp"],
                            )
                            # if abs(window[1] - window[0]) > (self.range[1] - self.range[0]):
                            # stack_val.append(row[2])
                            # stack_idx.append(row.Index)
                        # print(stack_val, stack_idx)

                    elif (window[0] <= self.range[1]) and (self.range[1] <= window[1]):
                        # print("stack start: case 2", idx)
                        if len(stack_val) == len(stack_idx) == 0:
                            stack_val.append(row[2])
                            stack_idx.append(row.Index)

                        else:
                            print(
                                "Something is wrong (start 2)",
                                self.mid,
                                window,
                                row.Index,
                                df.loc[row.Index, "timestamp"],
                            )

                    elif (window[0] >= self.range[1]) and (self.range[1] >= window[1]):
                        # print("stack stop: case 2", idx)
                        if len(stack_val) == len(stack_idx) == 0:
                            stack_val.append(row[2])
                            stack_idx.append(row.Index)
                            if len(stack_idx) >= time:
                                meta.append([stack_idx[0], stack_idx[-1]])
                            else:
                                num_lackpeak += 1
                        else:
                            print(
                                "Something is wrong (stop 2)",
                                self.mid,
                                window,
                                row.Index,
                                df.loc[row.Index, "timestamp"],
                            )
                        # print(stack_val, stack_idx)
                        stack_val = []
                        stack_idx = []

                    else:
                        if len(stack_val) == len(stack_idx) == 0:
                            stack_val.append(row[2])
                            stack_idx.append(row.Index)
                            # print("stack continue: ", idx)

                        else:
                            continue

            if len(stack_val) == len(stack_idx) == 0:
                # this is for the last stack which is detected as a peak
                if len(stack_idx) >= time:
                    meta.append([stack_idx[0], stack_idx[-1]])

            peak_idx = pd.Series(meta)
            peak_idx.columns = ["peak_idx"]
            self.num_lackpeak = num_lackpeak
            return peak_idx
        elif self.tt == "SMART_GLOVE_WRIST_DEVIATION_EG":
            df, ls = self.get_df()

            meta = []
            window = []
            window_idx = []
            stack_val = []
            stack_idx = []

            for row in df.itertuples():
                if len(window) < time:
                    # window에 element가 x개가 채워질 때까지는 그냥 채움
                    window.append(row[2])
                    window_idx.append(row.Index)
                    if len(window) == time:
                        if (
                            check_treasure_range(window, self.effective_range())
                            is True
                        ):
                            # stack_val.apppend(window)
                            stack_idx.append([window_idx[0], window_idx[-1]])
                else:
                    del window[0]
                    del window_idx[0]
                    window.append(row[2])
                    window_idx.append(row.Index)
                    # Case 1: 아예 처음으로 stack 시작
                    if check_treasure_range(window, self.effective_range()) is True:
                        if (len(stack_val) != 0) or (len(stack_idx) != 0):
                            # stack_val.append(window)
                            stack_idx.append([window_idx[0], window_idx[-1]])
                        # Case 2: 기존 stack에 이어서 붙여야 할 때
                        else:
                            if stack_idx[-1][0] < window_idx[0] < stack_idx[-1][1]:
                                # stack_val[-1].append(row[2])
                                del stack_idx[-1][-1]
                                stack_idx[-1].append(row.Index)
                            else:
                                # stack_val.append(window)
                                stack_idx.append([window_idx[0], window_idx[-1]])
                    # Case 3: Stack이 끝났을 때
                    else:
                        continue

            return pd.Series(stack_idx)

        else:
            print("Inappropriate game contents to detect peak (for now)")
            empty_list = []
            return empty_list

    def detect_modified_peak(self, filtering=1):

        """Visualize speed data as a plot (seaborn). 

        Args:
            sub: T/F. 훈련에 쓰이는 센서데이터 외 센서데이터 사용 여부 (e.g. POUR_WINE 에서 trans Forearm S/P 값 외의 센서값 사용 시 True 입력)
            filtering = 
            peak: int. peak detection algorithm 선택.

        Returns:
            seaborn plot. 
        """

        df_peak = self.detect_peak()
        df_acceleration = self.get_acceleration(filtering=filtering)
        if len(df_peak) < 1:
            return df_peak
        else:
            for index, value in df_peak.iteritems():
                i = value[0]
                if i != 0:
                    while np.sign(df_acceleration.loc[i]) == np.sign(
                        df_acceleration.loc[i - 1]
                    ):
                        i += 1
                        value.pop(0)
                        value.insert(0, i)

                else:
                    pass

                j = value[-1]
                if j != df_acceleration.index[-1]:
                    while np.sign(df_acceleration.loc[j]) == np.sign(
                        df_acceleration.loc[j + 1]
                    ):
                        j -= 1
                        value.pop(-1)
                        value.append(j)

                else:
                    pass

            return df_peak

    def detect_modified_s_peak(self, filtering = True):
        """Detect peaks by speed zero crossing algorithm in addition to the original algorithm.

        Args:
            filtering: butterworth low pass filter.
            
        Returns:
            Pandas Series consists of Peak indices (start, end). 
        """
        
        df_peak = self.detect_peak()
        df_speed = self.get_speed(filtering=filtering)
        if len(df_peak) < 1:
            return df_peak
        else:
            for index, value in df_peak.iteritems():
                i = value[0]
                if i != 0:
                    while np.sign(df_speed.loc[i]) == np.sign(df_speed.loc[i - 1]):
                        i += 1
                        value.pop(0)
                        value.insert(0, i)

                else:
                    pass

                j = value[-1]
                if j != df_speed.index[-1]:
                    while np.sign(df_speed.loc[j]) == np.sign(df_speed.loc[j + 1]):
                        j -= 1
                        value.pop(-1)
                        value.append(j)

                else:
                    pass

            return df_peak

    def detect_final_peak(self, filtering = False):

        """Detect peaks by averaging the results of speed zero crossing algorithm and acc zero crossing algorithm.

        Args:
            filtering: butterworth low pass filter.
            
        Returns:
            Pandas Series consists of Peak indices (start, end). 
        """

        
        df_mod_peak = self.detect_modified_peak(filtering=filtering)
        df_mod_s_peak = self.detect_modified_s_peak(filtering=filtering)
        if len(df_mod_peak) == len(df_mod_s_peak):
            df_final = pd.concat([df_mod_peak, df_mod_s_peak], axis=1)
            df_final["final"] = df_final.apply(
                lambda row: [
                    int((row[0][0] + row[1][0] + 1) / 2),
                    int((row[0][1] + row[1][1]) / 2),
                ],
                axis=1,
            )
        else:
            print("df_peak is not consistent")
        return df_final["final"]

    def detect_gap(self, mod=0):

        """Detect gaps in peak to peak. 

        Args:
            mod: int. peak detection algorithm

        Returns:
            Pandas Series with indices of gap (start, end). 
        """

        
        if mod == 0:
            df_peak = self.detect_final_peak(filtering=1)
        elif mod == 1:
            df_peak = self.detect_modified_peak()
        else:
            df_peak = self.detect_peak()

        meta_stack = []
        stack = []
        for index, value in df_peak.iteritems():
            if len(stack) == 0:
                stack.append(value[-1])
                continue
            elif len(stack) == 1:
                stack.append(value[0])
                meta_stack.append(stack)
                stack = []
                stack.append(value[-1])
                continue
            else:
                print("stacking error when finding gap")
        gap_idx = pd.Series(meta_stack)
        gap_idx.columns = ["gap_idx"]
        return gap_idx

    def peak_gap_info(self, p_or_g, mod=0):

        """Get list of indices, sensordata, speed, acceleration dataset of peaks and gaps. 

        Args:
            p_or_g: str. in ('p', 'peak', 'g', 'gap').
            mod: int. peak detection algorithm.

        Returns:
            Pandas DataFrame.
        """        

        if p_or_g == "p" or p_or_g == "peak":
            if mod == 0:
                series = self.detect_final_peak(filtering=1)
            elif mod == 1:
                series = self.detect_modified_peak()
            else:
                series = self.detect_peak()
        elif p_or_g == "g" or p_or_g == "gap":
            series = self.detect_gap(mod=mod)

        df_raw, ls = self.get_df()
        df_speed = self.get_speed()
        df_acceleration = self.get_acceleration()

        meta_degree = []
        meta_speed = []
        meta_acceleration = []

        for index, value in series.iteritems():
            ls_degree = []
            ls_speed = []
            ls_acceleration = []
            for i in range(value[0], int(value[-1] + 1)):
                ls_degree.append(df_raw.iloc[i, 1])
                ls_speed.append(df_speed[i])
                ls_acceleration.append(df_acceleration[i])
            meta_degree.append(ls_degree)
            meta_speed.append(ls_speed)
            meta_acceleration.append(ls_acceleration)

        df = pd.concat(
            [
                series,
                pd.Series(meta_degree),
                pd.Series(meta_speed),
                pd.Series(meta_acceleration),
            ],
            axis=1,
        )
        df.columns = ["idx", "angle", "speed", "acceleration"]

        return df

    def general_feature(self):

        """Calculate features for exercise mission result (unit of EMR). 

        Returns:
            dict. or dataframe consists of all game-level biomarkers
        """

        df_raw, ls = self.get_df()
        df_speed = self.get_speed(filtering=1)
        df_acc = self.get_speed(filtering=1)

        # 0. Preset
        target_playtime = {"target playtime": self.target_playtime}
        target_maintaintime = {"target maintain time": self.target_maintaintime}
        target_success_count = {"target success count": self.target_success_count}

        # 1. General Info
        playtime = {
            "playtime": round(
                (df_raw["timestamp"].max() - df_raw["timestamp"].min()).total_seconds(),
                2,
            )
        }
        playtime_record = {"playtime by record": self.playtime}
        playtime_ratio = {
            "playtime ratio": round((self.playtime / self.target_playtime), 2)
        }
        finished_way = {"finish type": self.finished_way()}
        feedback = {"feedback": self.feedback}

        # 2. AROM
        if self.tt == "SMART_GLOVE_FINGER_FLEX_EXT":
            calibrated_arom = {"Calibration AROM": "N/A"}
            success_arom = {"Success AROM": self.range}
            max_arom = {"Max AROM": "N/A"}
        else:
            calibrated_arom = {"Calibration AROM": self.cal_range[0]}
            success_arom = {"Success AROM": self.range}
            max_arom = {
                "Max AROM": [
                    np.percentile(df_raw[ls[1]][df_raw[ls[1]] < self.range[0]], 50),
                    np.percentile(df_raw[ls[1]][df_raw[ls[1]] > self.range[1]], 50),
                ]
            }

        # 3. Speed
        max_speed = {
            "Max Speed": [np.percentile(df_speed, 5), np.percentile(df_speed, 95)]
        }

        # 4. Coordination
        # 4-1. Motion Success
        success_count = {"Success Count": self.success_count}
        achieve_ratio = {"Achieve Ratio": self.achieve_ratio}
        earned_star = {"Star": self.earned_star}
        time_per_success = {"time per success": self.playtime / self.success_count}
        h_idx = {"H-index": self.h_index(thres_v=3, thres_t=3)}

        # 4-2. Smoothness
        nmu = {
            "No. Movement Unit": zero_crossing(df_acc, threshold=10)
            / self.success_count
        }
        log_jerk = {
            "Log Dimensionleimss Jerk": log_dimensionless_jerk(
                df_speed, 1 / self.timedelta
            )
        }
        sp_arc = {"SPARC": sparc(df_speed, 1 / self.timedelta)[0]}

        # 4-3. Correlation
        if self.tt == "SMART_GLOVE_FINGER_FLEX_EXT":
            df_raw["avg"] = df[ls[1:]].mean(axis=1)
            for i in range(1, 6):
                column = "avg" + str(i)
                df_raw[column] = ((5 * df_raw["avg"]) - df_raw[df_raw.columns[i]]) / 4
            finger_coordination = {
                "finger coordination": np.diag(df_raw.corr().iloc[0:5, -5:])
            }
        else:
            finger_coordination = {"finger coordination": "N/A"}

        # 5. Endurance

        feature_list = [
            target_playtime,
            target_maintaintime,
            target_success_count,
            playtime,
            playtime_record,
            playtime_ratio,
            finished_way,
            feedback,
            calibrated_arom,
            success_arom,
            max_arom,
            max_speed,
            success_count,
            achieve_ratio,
            earned_star,
            time_per_success,
            h_idx,
            nmu,
            log_jerk,
            sp_arc,
            finger_coordination,
        ]
        dic = {"Mission ID": self.mid}
        for d in feature_list:
            dic.update(d)

        if result == "df":
            return pd.DataFrame.from_dict(dic, orient="index")
        elif (result == "dic") or (result == "dict"):
            return dic
        else:
            return dic

    def get_peak_gap_feature(self, motion):

        """Calculate features for each motion (unit of motion). 

        Args:
            motion: in ("pp", "pm", "gi", "go")
            
        Returns:
            Pandas DataFrame consists of features of each motion. 
        """

        
        df_raw, ls = self.get_df()
        
        df_peak = self.peak_gap_info(p_or_g="p", mod=0)
        
        df_gap = self.peak_gap_info(p_or_g="g", mod=0)
        

        # 2. Peak idx info

        df_peak["len"] = df_peak["idx"].map(
            lambda x: (
                df_raw.loc[x[-1], "timestamp"] - df_raw.loc[x[0], "timestamp"]
            ).total_seconds()
        )
        

        # 2-1. Peak angle info

        df_peak["angle_median"] = df_peak["angle"].map(np.median)
        df_peak["angle_max"] = df_peak["angle"].map(
            lambda x: np.percentile(x, 95) if (np.mean(x) > 0) else np.percentile(x, 5)
        )
        df_peak["angle_iqr"] = df_peak["angle"].map(stats.iqr)
        df_peak["angle_kurtosis"] = df_peak["angle"].map(stats.kurtosis)

        # 2-2. Peak speed info
        df_peak["speed"] = df_peak["speed"].map(lambda x: reject_outliers(x, "speed"))
        df_peak["speed_median"] = df_peak["speed"].map(np.median)
        df_peak["speed_iqr"] = df_peak["speed"].map(stats.iqr)
        df_peak["speed_kurtosis"] = df_peak["speed"].map(stats.kurtosis)
        df_peak["speed_zero_cross"] = df_peak["speed"].map(zero_crossing)
        

        # 2-3. Peak acceleration info
        df_peak["acc_zero_cross"] = df_peak["acceleration"].map(zero_crossing)
        

        # 3. Gap idx info
        df_gap["len"] = df_gap["idx"].map(
            lambda x: (
                df_raw.loc[x[-1], "timestamp"] - df_raw.loc[x[0], "timestamp"]
            ).total_seconds()
        )
        
        # 3-1. Gap angle info
        # Seems no need for angle info of gaps

        # 3-2. gap speed info
        df_gap["speed"] = df_gap["speed"].map(lambda x: reject_outliers(x, "speed"))
        df_gap["speed_median"] = df_gap["speed"].map(np.median)
        df_gap["speed_max"] = df_gap["speed"].map(
            lambda x: np.percentile(x, 95) if (np.mean(x) > 0) else np.percentile(x, 5)
        )
        df_gap["speed_zero_cross"] = df_gap["speed"].map(zero_crossing)
        
        # 3-3. gap acceleration info
        df_gap["acc_zero_cross"] = df_gap["acceleration"].map(zero_crossing)
        
        # 4. Divide dataframe by motion and return
        df_peak["range"] = df_peak["angle"].map(np.mean)
        df_gap["start"] = df_gap["angle"].map(lambda x: x[0])
        
        if motion == "pp":
            df_peak_p = df_peak[df_peak["range"] >= self.range[1]].drop(
                columns=["range"]
            )
            
            return df_peak_p
        elif motion == "pm":
            df_peak_m = df_peak[df_peak["range"] <= self.range[0]].drop(
                columns=["range"]
            )
            return df_peak_m
        elif motion == "gi":
            # inner: pronation, flexion, R.D. / outer: supination, extension, U.D.
            df_gap_inner = df_gap[df_gap["start"] > (sum(self.range) / 2)].drop(
                columns=["start"]
            )
            return df_gap_inner
        elif motion == "go":
            df_gap_outer = df_gap[df_gap["start"] < (sum(self.range) / 2)].drop(
                columns=["start"]
            )
            return df_gap_outer

    def aggregate_feature(self, motion="all"):

        """Get aggregate statistics (centrality & variability) from EMR.
        
        Args:
            motion: in ('all', "pp", "pm", "gi", "go"). 특정 motion에 대한 feature만 필요한 경우 입력.

        Returns:
            Pandas DataFrame. aggregate features(statistics) of EMR.
        """

        
        if motion in ("pp", "pm", "gi", "go"):
            df = self.get_peak_gap_feature(motion)
            df_feature = df.describe().T
            df_feature["normality"] = (
                df.iloc[:, 4:].apply(stats.shapiro, axis=0).map(lambda x: x[1] >= 0.05)
            )
            df_feature["motion"] = motion
        else:
            ls = []

            for name in ["pp", "pm", "gi", "go"]:

                df = self.get_peak_gap_feature(name)

                df_feature_each = df.describe().T

                df_feature_each["normality"] = (
                    df.iloc[:, 4:]
                    .apply(stats.shapiro, axis=0)
                    .map(lambda x: x[1] >= 0.05)
                )

                df_feature_each["motion"] = name

                ls.append(df_feature_each)

            df_feature = pd.concat(ls)

        df_feature["iqr"] = df_feature["75%"] - df_feature["25%"]

        df_feature.rename(columns={"50%": "median"}, inplace=True)

        df_feature.drop(columns=["25%", "75%"], inplace=True)

        return df_feature

    def h_index(self, thres_v, thres_t):

        """number of maintaining attempts of under threshold speed with threshold time (in seconds)
        
        Args:
            thres_v: threshold speed to count as maintaining motion.
            thres_t: threshold time to count as an attempt

        Returns:
            h_idx (integer)
        """

        df_speed = self.get_speed(sub=0, filtering=1)
        thres_l = int(33.5 * thres_t)
        np_speed = df_speed.to_numpy()
        np_tf = abs(np_speed) < thres_v
        group = [(v, sum(1 for i in n)) for v, n in groupby(np_tf)]
        h_idx = 0
        for (tf, count) in group:
            if tf and count >= thres_l:
                h_idx += 1
            else:
                pass
        return h_idx

<IPython.core.display.Javascript object>